# Imports

In [1]:
import getpass
from pprint import pprint

import rhino_health as rh
from rhino_health.lib.endpoints.code_object.code_object_dataclass import (
    CodeObject,
    CodeObjectCreateInput,
    CodeObjectRunInput,
    CodeTypes
)
from rhino_health.lib.endpoints.sql_query.sql_query_dataclass import (
    SQLQueryImportInput,
    SQLQueryInput,
    SQLServerTypes,
    ConnectionDetails
)
from rhino_health.lib.endpoints.project.project_dataclass import ProjectCreateInput, 
from rhino_health.lib.endpoints.dataset.dataset_dataclass import DatasetCreateInput
from rhino_health.lib.endpoints.data_schema.data_schema_dataclass import DataSchemaCreateInput

/Users/drewafromsky/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Login and Create a Rhino Session
- Allows access to various endpoints directly from the RhinoSession
- Access endpoints at the following levels: `code_object`, `code_run`,`dataset`,`data_schema`,`federated_dataset`,`project`,`sql_query`,`user`, `workgroup`

In [3]:
my_username='drew@rhinohealth.com'

print("Logging In")
session = rh.login(
    username=my_username, 
    password=getpass.getpass(), 
    show_traceback=True,
    rhino_api_url='https://dev.rhinohealth.com/api/'
)
print("Logged In")

Logging In


 ········


Logged In


# Create a New Project

In [12]:
primary_workgroup_uid = session.current_user.primary_workgroup.uid
print(primary_workgroup_uid)

97d7456b-baef-4d88-bfb1-afd4975eaff2


In [10]:
# Input args for creating a new project with default permissions
new_project = ProjectCreateInput(
    session=session,
    name="Federated Datasets and Predictive Modeling (Setup Project + Collab) - drew@",
    description="Example Project for end-to-end Federated Modeling (project creation, \
                 add collaborator (invite + accept), Data Ext/Eng, Stat Analysis, training, and evaluation",
    type="Validation",
    primary_workgroup_uid=primary_workgroup_uid,
)

# Adds a new project owned by the currently logged in user.
project = session.project.add_project(new_project)

In [25]:
# Confirm project creation by listing all projects for the user
for project in session.project.get_projects():
    if project.name == new_project.name:
        print(project.name,"\n")
        print(project,"\n")
        project_uid = project.uid

Federated Datasets and Predictive Modeling (Setup Project + Collab) - drew@ 

Project session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10568fd60> creator_uid='e50f2461-3d7a-4fe0-b457-b2fe45d05532' created_at='2024-02-29T16:32:14.440680Z' name='Federated Datasets and Predictive Modeling (Setup Project + Collab) - drew@' description='Example Project for end-to-end Federated Modeling (project creation,                  add collaborator (invite + accept), Data Ext/Eng, Stat Analysis, training, and evaluation' type='Validation' primary_workgroup_uid='97d7456b-baef-4d88-bfb1-afd4975eaff2' permissions=None uid='1752fb63-c36c-4221-85a4-f979c5e921bb' slack_channel='' collaborating_workgroup_uids=['97d7456b-baef-4d88-bfb1-afd4975eaff2'] 



> Once the project has been created, navigate to the UI and the following should be displayed in the projects dashboard page and when clicking on the project we've just created

![Project Dashboard](./img/project_dashboard.png)

![Project Details](./img/project_details.png)

# Add a Collaborator

- Add New Collaborator
- Add _Rhino Sandbox Health System_
- Approve invitating selected collaborators to this project
- Add Collaborator to initiate the collaboration invitation process.

- You will receive approval within a few minutes. 

- You are now able to use datasets from the Rhino Sandbox Health System in addition to the datasets available in your Rhino Client.

- **Sandbox Meta Note**: We simulate having multiple datasets from two different organizations, also called the “cross-silo” setting in FL, by splitting the original MIMIC-IV dataset into multiple partitions. Each partition will represent the data from each organization. This is done purely for experimentation purposes. In real-world situations, there’s no need to partition data because each organization already has its own data, so the data is naturally partitioned.

In [55]:
def get_workgroup(name: str):
    """Get the workgroup metadata by name after listing all workgroups for collaboration"""

    result = None
    collaborating_workgroups = session.get(url=f"/projects/{project_uid}/workgroups_for_collaboration/")
    workgroups_metadata = collaborating_workgroups.raw_response.json()

    for workgroup_metadata in workgroups_metadata:
        if workgroup_metadata['name'] == name:
            result = workgroup_metadata
    
    if not result: return "Workgroup Name does not exist"
    else: return result

In [57]:
hco = get_workgroup(name="rhino-sandbox-hco")
hco

{'uid': '48cb366f-b05f-4ca2-8e1d-6dfc336cd344',
 'name': 'rhino-sandbox-hco',
 'organization': {'uid': 'e46a4068-a222-460f-9772-809d76415a43',
  'name': 'Rhino Sandbox'}}

In [58]:
aidev = get_workgroup(name="rhino-sandbox-aidev")
aidev

{'uid': '33cf1db0-de14-472a-8dcb-8d83de22d946',
 'name': 'rhino-sandbox-aidev',
 'organization': {'uid': 'e46a4068-a222-460f-9772-809d76415a43',
  'name': 'Rhino Sandbox'}}

In [ ]:
workgroups_metadata = collaborating_workgroups.raw_response.json()
workgroups_metadata

[{'uid': '54ed8ca7-9ac7-45b0-94fe-6ab081f57870',
  'name': 'MGH Radiology Dev',
  'organization': {'uid': '4325c725-a333-4c1b-9f97-1e5a24a56701',
   'name': 'Mass General Hospital'}},
 {'uid': 'caea1847-df51-4192-967f-06893399cd5c',
  'name': 'Cambridge Radiology',
  'organization': {'uid': '78a23702-ec4a-4c1d-913e-ba2be91df855',
   'name': 'Cambridge University Hospitals'}},
 {'uid': '5a2eee3b-dbf2-492d-93fa-1175ae2ad7a6',
  'name': 'MGH Radiology',
  'organization': {'uid': '1d6e4314-5fe3-43d2-966c-9c11bb2662b8',
   'name': 'Massachusetts General Hospital'}},
 {'uid': '6661e36f-00f3-4e69-9f3d-4e6ea4f43be6',
  'name': 'Lahey Radiology',
  'organization': {'uid': '71c70b1e-bc03-4a57-aa70-6f5bd603a13a',
   'name': 'Lahey Hospital & Medical Center'}},
 {'uid': '2466f9a0-fd16-44fd-8451-06a01ec6ed9a',
  'name': 'DASA Radiology',
  'organization': {'uid': 'f9a33fc2-beb3-4a79-b75c-9dfa8097a3d5',
   'name': 'DASA'}},
 {'uid': 'fbb8d2e0-e513-429e-ba76-514302e20685',
  'name': 'SNUH Radiology',

In [ ]:
add_aidev_collaborator = session.project.add_collaborator(
    project_uid=project_uid,
    collaborating_workgroup_uid = aidev.get('uid')
)

In [ ]:
add_hco_collaborator = session.project.add_collaborator(
    project_uid=project_uid,
    collaborating_workgroup_uid = hco.get('uid')
)

In [65]:
# List collaborating work groups for the current project
session.project.get_collaborating_workgroups(project_uid)

[Workgroup(session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10568fd60>, uid='97d7456b-baef-4d88-bfb1-afd4975eaff2', name='Rhino Health Test', org_name='Rhino Health Test'),
 Workgroup(session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10568fd60>, uid='33cf1db0-de14-472a-8dcb-8d83de22d946', name='rhino-sandbox-aidev', org_name='Rhino Sandbox'),
 Workgroup(session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10568fd60>, uid='48cb366f-b05f-4ca2-8e1d-6dfc336cd344', name='rhino-sandbox-hco', org_name='Rhino Sandbox')]

In [80]:
aidev_workgroup_uid = aidev.get('uid')

is_aidev_collaborator_approved = session.get(
    url=f"/projects/{project_uid}/collaborator/{aidev_workgroup_uid}/is_collaborator_approved/"
).raw_response.json()

is_aidev_collaborator_approved

{'status': 'success', 'is_policy_approved': False}

In [81]:
hco_workgroup_uid = hco.get('uid')

is_hco_collaborator_approved = session.get(
    url=f"/projects/{project_uid}/collaborator/{hco_workgroup_uid}/is_collaborator_approved/"
).raw_response.json()

is_hco_collaborator_approved  # is_policy_approved: True --> after a collaborator has approved the permissions policy

{'status': 'success', 'is_policy_approved': True}

In [79]:
all_project_collaborators = session.get(
    url=f"/projects/{project_uid}/collaborators/"
).raw_response.json()

all_project_collaborators

[{'uid': '97d7456b-baef-4d88-bfb1-afd4975eaff2',
  'name': 'Rhino Health Test',
  'organization': {'uid': '6a1efb12-0636-4c11-8979-3844d9c09f70',
   'name': 'Rhino Health Test'}},
 {'uid': '33cf1db0-de14-472a-8dcb-8d83de22d946',
  'name': 'rhino-sandbox-aidev',
  'organization': {'uid': 'e46a4068-a222-460f-9772-809d76415a43',
   'name': 'Rhino Sandbox'}},
 {'uid': '48cb366f-b05f-4ca2-8e1d-6dfc336cd344',
  'name': 'rhino-sandbox-hco',
  'organization': {'uid': 'e46a4068-a222-460f-9772-809d76415a43',
   'name': 'Rhino Sandbox'}}]